In [1]:
import requests
from bs4 import BeautifulSoup

ilce_listesi = {
    "5785": "Arnavutköy", "434": "Avcılar", "437": "Bağcılar", "438": "Bahçelievler",
    "435": "Bakırköy", "5786": "Başakşehir", "436": "Bayrampaşa", "439": "Beşiktaş",
    "1077": "Beylikdüzü", "440": "Beyoğlu", "441": "Büyükçekmece", "442": "Çatalca",
    "445": "Eminönü", "443": "Esenler", "5787": "Esenyurt", "444": "Eyüp", "446": "Fatih",
    "447": "Gaziosmanpaşa", "448": "Güngören", "449": "Kağıthane", "450": "Küçükçekmece",
    "5677": "MimarSinan/Büyükçekmece", "451": "Sarıyer", "452": "Silivri", "5765": "Sultangazi",
    "453": "Şişli", "454": "Zeytinburnu", "455": "Adalar", "1722": "Ataşehir", "456": "Beykoz",
    "1244": "Çekmeköy", "457": "Kadıköy", "458": "Kartal", "459": "Maltepe", "460": "Pendik",
    "5788": "Sancaktepe", "461": "Sultanbeyli", "462": "Şile", "463": "Tuzla", "464": "Ümraniye",
    "465": "Üsküdar"
}

def get_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

for ilce_numarasi, ilce_ismi in ilce_listesi.items():
    toplam_ilan_sayisi = 0
    toplam_basvuru_sayisi = 0

    base_url = f"https://www.kariyer.net/is-ilanlari/{ilce_ismi}?ct=34,82&tt={ilce_numarasi}&date=3g"
    html = get_html(base_url)
    
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        sayfa_numarasi_element = soup.select_one("button.page-link[aria-checked='true']")
        sayfa_sayisi = 1
        if sayfa_numarasi_element and sayfa_numarasi_element.get("aria-setsize"):
            sayfa_sayisi = int(sayfa_numarasi_element.get("aria-setsize", "1"))

        for sayfa_no in range(1, sayfa_sayisi + 1):
            page_url = f"{base_url}&cp={sayfa_no}"
            page_html = get_html(page_url)
            
            if page_html:
                page_soup = BeautifulSoup(page_html, 'html.parser')
                ilan_kartlari = page_soup.select("a.k-ad-card")

                if ilan_kartlari:
                    ilan_sayisi = len(ilan_kartlari)
                    toplam_ilan_sayisi += ilan_sayisi

                    for ilan_karti in ilan_kartlari:
                        ilan_detay_url_relative = ilan_karti.get("href")
                        ilan_detay_url_absolute = f"https://www.kariyer.net{ilan_detay_url_relative}"

                        ilan_detay_html = get_html(ilan_detay_url_absolute)
                        
                        if ilan_detay_html:
                            ilan_detay_soup = BeautifulSoup(ilan_detay_html, 'html.parser')
                            
                            # Find the application count
                            basvuru_sayisi_tagi = ilan_detay_soup.select_one("p.mb-0 span[data-v-5fb4fcba='']")
                            
                            if basvuru_sayisi_tagi:
                               basvuru_sayisi_text = basvuru_sayisi_tagi.parent.text.strip()
                               basvuru_sayisi_sadece_rakamlar = ''.join(filter(str.isdigit, basvuru_sayisi_text))

                            try:
                               basvuru_sayisi_int = int(basvuru_sayisi_sadece_rakamlar)
                               toplam_basvuru_sayisi += basvuru_sayisi_int
                            except ValueError:
                               pass
                
            else:
                print(f"Error fetching page {page_url}")
        
        print(f"İlçe İsmi: {ilce_ismi}")
        print(f"Toplam İlan Sayısı: {toplam_ilan_sayisi}")
        print(f"Toplam Başvuru Sayısı: {toplam_basvuru_sayisi}")
        print()
    else:
        print(f"Error fetching base URL {base_url}")
